In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## NewYork City Bike

在预测模型训练之前对数据进行清洗， 数据清洗的代码已经整理在 `citibikeUtil.py`中，预测器的代码已经整理在`predictor.py`中，
`citybikeDraw.py`是用来生成图片的文件 `citybikeMetrics.py`计算指标

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd

In [4]:
from citybikeUtil import CB201808, CB201809, CB201810, CB201811, CB201812, CB201901, CB201902
from predictor import LSTMPredictor, DecisionTreePredictor, XGBoostPredictor

Using TensorFlow backend.


## LSTMPredictor

In [5]:
lstm_predictor = LSTMPredictor()
lstm_predictor.train_month = CB201808

### 训练模型

In [6]:
train_x, train_y = lstm_predictor.get_train_vec(CB201808)
test_x, test_y = lstm_predictor.get_train_vec(CB201809)
lstm_predictor.train_model(train_x, train_y)
predict_y = lstm_predictor.model.predict(test_x)

./201808-citibike-tripdata.csv.lstm.h5


### 导出PMML

In [7]:
lstm_predictor.keras2pmml(estimator=lstm_predictor.model, transformer=None, file='keras.pmml')

./pickles/citybick/CB201808.pmml True

    [x] Model validation successful.
    [x] Generating Data Dictionary:
        [-] x0...OK!
        [-] x1...OK!
        [-] x2...OK!
        [-] x3...OK!
        [-] x4...OK!
        [-] x5...OK!
        [-] x6...OK!
        [-] x7...OK!
        [-] x8...OK!
        [-] x9...OK!
        [-] x10...OK!
        [-] x11...OK!
    
save file keras.pmml


### 根据预测结果的个数生成 时间间隔 

预测月份是9月份 因为需要生成13点的向量 需要收集前12点的数据 所以从13点开始 看到预测结果的个数是707 所以periods参数为707 表示生成707个周期的结果 freq代表频率 'H'代表小时 每小时生成一个

### 使用seaborn画图需要将数据合成df

In [ ]:
date_index = pd.date_range(start='2018-09-01 13:00:00', periods=707, freq='H')
merge_data = {
    'predict': predict_y.flatten(),
    'real': test_y.flatten()
}
lstm_df = pd.DataFrame(merge_data, index=date_index)

In [ ]:
date_index.tolist()

In [ ]:
import json

In [ ]:
merge_data = {
    'predict': predict_y.flatten().tolist(),
    'real': test_y.flatten().tolist(),
    "time": date_index.tolist()
}

# 实验结果

## 预测结果图展示
因为一个月的太长了，只显示调整为1周

In [ ]:
sns.set(style="whitegrid")
sns.set_context(rc={"lines.linewidth": 2})

In [ ]:
# plt.figure(figsize=(15, 5))
f, ax = plt.subplots(figsize = (8, 4))
# 设置图名称
# ax.set_title('DataStream DisOrder Predict - 1st Week', fontsize=15)
# 设置X轴 Y轴 的label
ax.set_xlabel('Time',fontsize=10)
ax.set_ylabel('Disorder Rate',fontsize=15)
sns.lineplot(size='event',
             markers=True,dashes=False,
             data=lstm_df[12:92])
plt.savefig('lstm-bike.eps',bbox_inches='tight',format='eps')

In [ ]:
# plt.figure(figsize=(15, 5))
import matplotlib
# matplotlib.rcParams['font.sans-serif'] = ['Microsoft Yahei']
f, ax = plt.subplots(figsize = (20, 3))
# 设置图名称
# ax.set_title('DataStream DisOrder Predict - 1st Week', fontsize=15)
# 设置X轴 Y轴 的label
# ax.set_xlabel('时间',fontsize=15)
# ax.set_ylabel('数据乱序率',fontsize=20)
# plt.xlabel('星期',fontdict={'family':'SimSun', 'size': 14})
plt.ylabel('数据乱序率',fontdict={'family':'SimSun', 'size': 20})
sns.lineplot(size='event',
             markers=True,dashes=False,
             data=lstm_df[:169])
plt.savefig('单车预测结果-lstm.pdf',bbox_inches='tight',format='pdf')

## 导出eps

## 封装成一个画图类

In [ ]:
from citybikeDraw import DrawLSTM
draw = DrawLSTM()
draw.set_train_month(CB201808)
test_y, predict_y = draw.get_predict_result(CB201808, CB201809)

In [ ]:
draw.draw_real_predict(
    week_num=1,
    start_time="2018-09-01 13:00:00",
    title="DataStream DisOrder Predict",
    figsize=(15,5)
)

## 回归性能指标

分月份展示:

比如训练 CB201808 预测 CB201809  均方根误差（标准误差）平均绝对误差  R-squared

比如训练 CB201809 预测 CB201810 均方根误差（标准误差）平均绝对误差  R-squared

### 均方根误差(Root Mean Squard Error，RMSE)

$RMSE=\sqrt{\frac{1}{m} \sum_{i=1}^{m}\left(f_{i}-y_{i}\right)^{2}}$

标准差是方差的算术平方根。标准误差是均方误差的算术平方根。
标准差是用来衡量一组数自身的离散程度，而均方根误差是用来衡量观测值同真值之间的偏差，它们的研究对象和研究目的不同，但是计算过程类似

它的意义在于开个根号后，误差的结果就与数据是一个级别的，可以更好地来描述数据。标准误差对一组测量中的特大或特小误差反映非常敏感，所以，标准误差能够很好地反映出测量的精密度。这正是标准误差在工程测量中广泛被采用的原因。

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
np.sqrt(mean_squared_error(test_y, predict_y))

### 平均绝对误差(Mean Absolute Error，MAE)

$MAE=\frac{1}{m} \sum_{i=1}^{m}\left|f_{i}-y_{i}\right|$

平均绝对误差能更好地反映预测值误差的实际情况.

In [ ]:
from sklearn.metrics import median_absolute_error
median_absolute_error(test_y, predict_y)

### R-squared
$R^{2}=1-\frac{\sum_{i=1}^{m}\left(f_{i}-y_{i}\right)^{2}}{\sum_{i=1}^{m}\left(\overline{y_{i}}-y_{i}\right)^{2}}$

上面分子就是我们训练出的模型预测的误差和。
下面分母就是瞎猜的误差和。（通常取观测值的平均值）

如果结果是0，就说明我们的模型跟瞎猜差不多。
如果结果是1。就说明我们模型无错误。

$R^2$介于0~1之间，越接近1，回归拟合效果越好，一般认为超过0.8的模型拟合优度比较高。


In [ ]:
from sklearn.metrics import r2_score
r2_score(test_y, predict_y)

## 统计每个月预测结果的回归性能指标

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

In [ ]:
from citybikeMetrics import get_reg_metrics

In [ ]:
months = [CB201808, CB201809, CB201810, CB201811, CB201812, CB201901, CB201902]

### 使用更新的模型

In [ ]:
update_metrics = get_reg_metrics(months, True)

### 不使用更新的模型

In [ ]:
unupdate_metrics = get_reg_metrics(months, False)

In [ ]:
result = pd.concat([update_metrics,unupdate_metrics], axis=1)

In [ ]:
result.columns = [["Update Model","Update Model","Update Model","UnUpdate","UnUpdate","UnUpdate"],["RMSE", "MAE", "R2S","RMSE", "MAE", "R2S"]]

In [ ]:
result

In [ ]:
update_metrics

In [ ]:
unupdate_metrics